In [1]:
import pandas as pd
import numpy as np

In [4]:
c=pd.read_json("results_IR.json")

In [221]:
c

,qid:,qstr,candidates_exist,candidates_tfidf,candidates_bm25
0,1,covid-19 OR sars-cov-2,"[M33634079, M33457109, M33602867, M33585567, M...","[BASE::ftzenodo:oai:zenodo.org:4017260, BASE::...",[BASE::ftarxivpreprints:oai:arXiv.org:2009.029...
1,2,dementia,"[M31476229, M29425707, M30482323, M27643909, M...","[M23074509, BASE::ftdoajarticles:oai:doaj.org/...","[M9056119, M1595095, M31236955, M33158014, BAS..."
2,3,biodiversity,"[M31517113, M32158136, M31749770, M31219416, M...","[B4460001, BASE::ftdoajarticles:oai:doaj.org/a...","[FRL6403566, BASE::ftifpriir:oai:ebrary.ifpri...."
3,4,influenza,"[M32048225, M31478058, M28869984, M28302313, M...","[M33131228, M33297613, BASE::ftrobertkoch:oai:...","[COVID19::1eef8b87c14fa0a85b36fcb99e33aa1d0, C..."
4,5,malaria,"[M30550661, M24399475, M24109136, M23953767, M...","[M26702637, BASE::ftdoajarticles:oai:doaj.org/...","[M23294805, BASE::ftwhoiris:oai:localhost:1066..."
...,...,...,...,...,...
995,1446,epigenetik,"[M25446312, M24337072]","[PSYNDEX0283783, THIEME10_1055_s_0032_1332815,...","[PSYNDEX0252889, PSYNDEX0232492, PSYNDEX031655..."
996,1452,stroke AND unit,"[M33389780, M32115340, M32818753, M32347338, M...","[M32324916, M31008352, BASE::ftdoajarticles:oa...","[M32324916, BASE::ftdoajarticles:oai:doaj.org/..."
997,1453,praxis AND der AND intensivmedizin,"[M32086542, M26863643, M32870328, M23868519]","[THIEME10_1055_s_2000_8048, THIEME10_1055_s_20...","[PSYNDEX0330612, HECLINETHN57244, THIEME10_105..."
998,1454,das AND gesundheitssystem AND in AND deutschland,"[M21800282, M19444774, M29741426, M33288301, M...","[BASE::ftwikibooks:dewikibooks:31258:109386, B...","[SOMED644637, PSYNDEX0322470, ZBW590987062, HE..."


In [238]:
##### I am selecting some subset of the data the reqired input is for different models there should be a column including 
# list of results for each model and each query. 
c_alternative=c.loc[0:120];

In [211]:
def give_points(x):
    """ Documents are listed in decreasing relevance so function giving every each of the document inside of the list a
    relevance point based on their ranking in the list first one gets 10 and the last one 1 if a list has more than 10 
    items only the first 10 will be taken account"""
    dicti={}
    sayac=10
    for i in x:
        dicti[i]= sayac
        sayac-=1
        if sayac <=0:
            break
    return dicti
def calculate_DCG(liste):
    """Applies DCG based on the list that has been given """
    sumv=0
    for i in liste:
        sumv+=((i)/np.log2(list(liste).index(i)+2))
    return sumv

In [224]:
c_alternative["candidates_exist"].apply(lambda x: give_points(x))

0     {'M33634079': 10, 'M33457109': 9, 'M33602867':...
1     {'M31476229': 10, 'M29425707': 9, 'M30482323':...
2     {'M31517113': 10, 'M32158136': 9, 'M31749770':...
3     {'M32048225': 10, 'M31478058': 9, 'M28869984':...
4     {'M30550661': 10, 'M24399475': 9, 'M24109136':...
5     {'M33178181': 10, 'M33476812': 9, 'M31455506':...
6     {'M31940869': 10, 'M33533750': 9, 'M28289424': 8}
7     {'M33414994': 10, 'M32291236': 9, 'M33483615':...
8     {'M31642554': 10, 'M32229385': 9, 'M33575938':...
9     {'M11887331': 10, 'M2134675': 9, 'M1186903': 8...
10                                    {'M31158670': 10}
11    {'M33557033': 10, 'M33427247': 9, 'M32129917':...
12                                    {'M32017810': 10}
13                                    {'M32065556': 10}
14    {'M24603947': 10, 'M24393820': 9, 'M22252595':...
15    {'M30550648': 10, 'M32565763': 9, 'M32635652':...
16                                                   {}
17    {'M33419321': 10, 'M32824581': 9, 'M329022

In [242]:
# first using give_points function to give relevance point to each candidate documents in descending order real includes 
# best alignment order.
# Will give a division zero error if the result list of a model has no object inside.

real=c_alternative["candidates_exist"].apply(lambda x: give_points(x))

# then creating the dicti empty dictionary. Then vectorizing results of models acording to the points of best alignment. If 
# model has a non relevant document in list for that object of list 0 point returned but if an object matches in real object
# it returns the relevance point for that object and therefore we have the proper structure to apply DCG on it. 

# To apply this results to another model just change the result column in for loop and it will do the rest

dicti={}
for k in c_alternative.candidates_tfidf.index:
    dicti[k]=[]
    for i in c_alternative.loc[k].candidates_tfidf:
        if i in real[k]:
            dicti[k].append(real[k][i])
        else:
            dicti[k].append(0)
result = {}

# Creating another result dictionary and computing and assigning de nCDG values for each index. For NCDG = DCG/ IDCG the CDG is
# the DCG is the computed by the vector of the result and IDCG is applying DCG to the best result. 

for i in dicti.keys():
    result[i] = calculate_DCG(dicti[i])/calculate_DCG(real[i].values())
result

C:\Users\silen\AppData\Local\Temp\ipykernel_2224\3257849000.py:27: RuntimeWarning: invalid value encountered in double_scalars
  result[i] = calculate_DCG(dicti[i])/calculate_DCG(real[i].values())


{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.1518264099903325,
 8: 0.01929278331841544,
 9: 0.0,
 10: 0.43067655807339306,
 11: 0.05501204432532872,
 12: 0.3010299956639812,
 13: 0.3868528072345416,
 14: 0.0,
 15: 0.0,
 16: nan,
 17: 0.0,
 18: 0.0,
 19: nan,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: nan,
 24: 0.0,
 25: 0.11350279283866517,
 26: nan,
 27: 0.0,
 28: 0.0,
 29: 0.07031977198449162,
 30: 0.0,
 31: 0.4061680417033703,
 32: 0.47626231216265874,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.22596666707058213,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: nan,
 51: 0.0,
 52: nan,
 53: 0.0,
 54: 0.24674303639894693,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.4061680417033703,
 59: 0.0,
 60: 0.0,
 61: nan,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: nan,
 67: 0.0,
 68: 0.03188978716612874,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 

In [241]:
c.loc[120:130]

,qid:,qstr,candidates_exist,candidates_tfidf,candidates_bm25
120,135,gesundheitskompetenz,"[M30176683, M31950230, M24671890, M31307911, M...","[THIEME10_1055_s_0028_1086260, THIEME10_1055_s...","[PSYNDEX0327154, PSYNDEX0339605, THIEME10_1055..."
121,136,pulvinaria,[],[],"[M30582471, M10746665, THIEME10_1055_s_0032_13..."
122,137,cannabis,"[M32585801, M32714224, M31096702, M30426073, N...",[BASE::ftdoajarticles:oai:doaj.org/article:8ae...,"[M12681525, M27262964, M28733813, M30302367, M..."
123,138,pharmakologie,[M22230966],"[HECLINETHN43340, BFR389977349, PSYNDEX0081769...","[SKH389240036, HECLINETHN43340, BFR389977349, ..."
124,139,schlaf,"[M32431564, M32274782, M28086246, M30141742, M...","[THIEME10_1055_s_2005_867168, PSYNDEX0346046, ...","[PSYNDEX0006629, PSYNDEX0346046, PSYNDEX023696..."
125,140,trauma,"[M18655938, NLM101550300, M33612396, M30909889...","[BASE::ftwikibooks:itwikibooks:12350:46097, M2...","[M20600466, BASE::ftncaccalio:oai:calio.dspace..."
126,141,schizophrenia,"[M32865492, M27189524, M25611712, NLM101544147...","[M32049268, M33837784, M32848912, M33657348, M...","[M10789350, M29555214, M32498652, M31954758, M..."
127,143,thromboseprophylaxe,"[M32018283, M22504622, M23404249, M22504621, M...","[THIEME10_1055_s_2004_822665, THIEME10_1055_s_...","[CC_BIBNET_bibnet9098, THIEME10_1055_s_2003_40..."
128,145,gesundheitskompetenz AND health AND literacy,"[M30176683, M32100565, M33278318, M33422042, M...","[PSYNDEX0327154, M26571285, THIEME10_1055_s_00...","[PSYNDEX0327154, PSYNDEX0302181, PSYNDEX033960..."
129,146,sterbehilfe,"[M33049787, M27119531, M26577148, M19004177, M...","[THIEME10_1055_a_1235_6550, PSYNDEX0164610, TH...","[HECLINETHN212910, THIEME10_1055_a_1235_6550, ..."
